# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,remire-montjoly,4.9167,-52.2667,23.02,100,20,1.03,GF,1712568787
1,1,wailua homesteads,22.0669,-159.3780,22.44,84,75,7.20,US,1712568592
2,2,kayangel,8.0823,134.7173,28.25,77,15,6.12,PW,1712568787
3,3,kodiak,57.7900,-152.4072,-1.27,86,0,0.00,US,1712568787
4,4,alice springs,-23.7000,133.8833,24.56,21,99,3.09,AU,1712568754


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:



# Configure the map plot with different colors for each city
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', size='Humidity', geo=True, tiles='OSM', title='City Map with Humidity', color='City', width=1000, height=600)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Define the criteria
max_temp_lower = 27
max_temp_higher = 21
max_wind_speed = 4.5
cloudiness = 0

# Filter the DataFrame based on the criteria
filter_weather_df = city_data_df[
    (city_data_df['Max Temp'] < max_temp_lower) & 
    (city_data_df['Max Temp'] > max_temp_higher) & 
    (city_data_df['Wind Speed'] < max_wind_speed) & 
    (city_data_df['Cloudiness'] == cloudiness)
]
# Drop any rows with null values
filter_weather_df = filter_weather_df.dropna()
# Display sample data
filter_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,119,apatzingan,19.0833,-102.3500,22.52,41,0,0.95,MX,1712568811
211,211,mount isa,-20.7333,139.5000,24.87,41,0,3.60,AU,1712568829
251,251,vila velha,-20.3297,-40.2925,22.19,95,0,1.54,BR,1712568295
324,324,maceio,-9.6658,-35.7353,24.69,94,0,1.54,BR,1712568727
325,325,hotan,37.0997,79.9269,22.25,9,0,2.98,CN,1712568851
337,337,pisco,-13.7000,-76.2167,21.03,83,0,1.54,PE,1712568846
369,369,carros,43.7894,7.1840,21.54,50,0,3.09,FR,1712568861
385,385,gustavia,17.8962,-62.8498,25.79,83,0,0.89,BL,1712568759
413,413,itaguai,-22.8522,-43.7753,21.79,94,0,2.57,BR,1712568725
431,431,bakau,13.4781,-16.6819,26.08,57,0,4.12,GM,1712568873


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filter_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
119,apatzingan,MX,19.0833,-102.3500,41,
211,mount isa,AU,-20.7333,139.5000,41,
251,vila velha,BR,-20.3297,-40.2925,95,
324,maceio,BR,-9.6658,-35.7353,94,
325,hotan,CN,37.0997,79.9269,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000  # Set the radius to 10,000 meters (10 kilometers)
params = {
    "radius": radius, 
    "category": "accommodation.hotel", 
    "api_key": "geoapify_key"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    latitude = row['Lat']
    longitude = row['Lng']
   
    params["filter"] = f"radius_{radius}.hotel"
    params["bias"] = f"point({longitude},{latitude})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v1/places"

    
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response_json = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = response_json['features'][0]['properties']['name']
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
   
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
apatzingan - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
maceio - nearest hotel: No hotel found
hotan - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
carros - nearest hotel: No hotel found
gustavia - nearest hotel: No hotel found
itaguai - nearest hotel: No hotel found
bakau - nearest hotel: No hotel found
nieuw amsterdam - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
qrendi - nearest hotel: No hotel found
san ignacio - nearest hotel: No hotel found
gargalianoi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
119,apatzingan,MX,19.0833,-102.3500,41,No hotel found
211,mount isa,AU,-20.7333,139.5000,41,No hotel found
251,vila velha,BR,-20.3297,-40.2925,95,No hotel found
324,maceio,BR,-9.6658,-35.7353,94,No hotel found
325,hotan,CN,37.0997,79.9269,9,No hotel found
337,pisco,PE,-13.7000,-76.2167,83,No hotel found
369,carros,FR,43.7894,7.1840,50,No hotel found
385,gustavia,BL,17.8962,-62.8498,83,No hotel found
413,itaguai,BR,-22.8522,-43.7753,94,No hotel found
431,bakau,GM,13.4781,-16.6819,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot with custom hover message
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',
    geo=True,
    tiles='OSM',
    title='City Map with Humidity',
    color='City',
    hover_cols=['City', 'Country', 'Hotel Name'],
    width=1000,
    height=600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)